<a href="https://colab.research.google.com/github/piyush-98/emotion_detector_text/blob/master/emotion_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!cp "/content/drive/My Drive/merged_training.pkl" "/content/"

In [4]:
!ls

drive  merged_training.pkl  sample_data


In [2]:
import pandas as pd
import pickle

In [3]:
data = pd.read_pickle(r'merged_training.pkl')

In [4]:
data.shape

(416809, 2)

In [5]:
data.columns = ['content', 'sentiments']

In [6]:
import re
def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [7]:
data=data.iloc[:50000]

In [8]:
#Making all letters lowercase
import nltk
nltk.download('wordnet')
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing Punctuation, Symbols
data['content'] = data['content'].str.replace('[^\w\s]',' ')
#Removing Stop Words using NLTK
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Lemmatisation
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'] = data['content'].apply(lambda x: " ".join(de_repeat(x) for x in x.split()))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
data.shape

(50000, 2)

In [10]:
#Encoding output labels 'sadness' as '1' & 'happiness' as '0'
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.sentiments.values)
# Splitting into training and testing data in 90:10 ratio

In [11]:
!pip install transformers

In [12]:
from transformers import BertTokenizer, TFBertModel

In [13]:
import tensorflow as tf

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
input_ids = tf.constant(tokenizer.encode("Hell is very bad"))[None, :]  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]


In [16]:
last_hidden_states.shape

TensorShape([1, 6, 768])

In [ ]:
def getbertEmbeddings(X):
    ls=[]
    for ix in range(X.shape[0]):
        sent = X['content'].iloc[ix]
        input_ids = tf.constant(tokenizer.encode(sent))[None, :]
        outputs = model(input_ids)
        last_hidden_states = outputs[0]
        last_hidden_states = last_hidden_states[0]
        ls.append(last_hidden_states)  
    return ls

In [36]:
import numpy as np
bert_data=getbertEmbeddings(data.iloc[:1000])

KeyboardInterrupt: ignored

In [37]:
bert_data_arr=np.array(bert_data)

In [38]:
bert_data_arr.shape

(1000,)

In [68]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x=pad_sequences(bert_data_arr,maxlen=20,padding='post',dtype='float64')

In [74]:
x.shape

(1000, 20, 768)

In [76]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential

In [77]:
classifier = Sequential()
classifier.add(LSTM(128,input_shape=(20,768),return_sequences=True))
classifier.add(Dropout(0.5))
classifier.add(LSTM(128))
classifier.add(Dropout(0.5))
classifier.add(Dense(6))
classifier.add(Activation('softmax'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           459264    
_________________________________________________________________
dropout_37 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_38 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 774       
_________________________________________________________________
activation_10 (Activation)   (None, 6)                 0         
Total params: 591,622
Trainable params: 591,622
Non-trainable params: 0
__________________________________________________

In [78]:
classifier.compile( loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'] )

In [86]:
#Encoding output labels 'sadness' as '1' & 'happiness' as '0'
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.iloc[:1000].sentiments.values)
# Splitting into training and testing data in 90:10 ratio

In [87]:
from keras.utils import to_categorical
y_label=to_categorical(y)

In [88]:
y_label.shape

(1000, 6)

In [90]:
classifier.fit(x,y_label,batch_size=100,epochs=20)

Epoch 1/20
10/10 [==============================] - 1s 147ms/step - loss: 1.2495 - accuracy: 0.5580
Epoch 2/20
10/10 [==============================] - 1s 148ms/step - loss: 1.1997 - accuracy: 0.5770
Epoch 3/20
10/10 [==============================] - 1s 147ms/step - loss: 1.0668 - accuracy: 0.6170
Epoch 4/20
10/10 [==============================] - 1s 147ms/step - loss: 0.9471 - accuracy: 0.6730
Epoch 5/20
10/10 [==============================] - 1s 147ms/step - loss: 0.8280 - accuracy: 0.7340
Epoch 6/20
10/10 [==============================] - 1s 148ms/step - loss: 0.6192 - accuracy: 0.7970
Epoch 7/20
10/10 [==============================] - 1s 149ms/step - loss: 0.4662 - accuracy: 0.8590
Epoch 8/20
10/10 [==============================] - 1s 146ms/step - loss: 0.3419 - accuracy: 0.9010
Epoch 9/20
10/10 [==============================] - 1s 147ms/step - loss: 0.2611 - accuracy: 0.9290
Epoch 10/20
10/10 [==============================] - 1s 145ms/step - loss: 0.2148 - accuracy: 0.9330